In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Peach___Bacterial_spot', 'Peach___healthy']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(11337, 128, 128, 3)

In [8]:
y.shape

(11337, 6)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(9069, 128, 128, 3)

In [11]:
X_test.shape

(2268, 128, 128, 3)

In [12]:
y_train.shape

(9069, 6)

In [13]:
y_test.shape

(2268, 6)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

6

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 12:25:52.017814: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


284/284 [==============================] - 145s 506ms/step - loss: 0.6161 - accuracy: 0.7778 - val_loss: 1.1077 - val_accuracy: 0.7302
Epoch 2/10
284/284 [==============================] - 141s 498ms/step - loss: 0.2758 - accuracy: 0.9090 - val_loss: 0.3380 - val_accuracy: 0.9021
Epoch 3/10
284/284 [==============================] - 129s 454ms/step - loss: 0.1877 - accuracy: 0.9370 - val_loss: 0.1486 - val_accuracy: 0.9466
Epoch 4/10
284/284 [==============================] - 136s 479ms/step - loss: 0.1479 - accuracy: 0.9483 - val_loss: 0.2892 - val_accuracy: 0.9048
Epoch 5/10
284/284 [==============================] - 89s 314ms/step - loss: 0.1245 - accuracy: 0.9570 - val_loss: 0.5251 - val_accuracy: 0.8488
Epoch 6/10
284/284 [==============================] - 90s 316ms/step - loss: 0.0967 - accuracy: 0.9677 - val_loss: 0.1339 - val_accuracy: 0.9599
Epoch 7/10
284/284 [==============================] - 126s 444ms/step - loss: 0.0911 - accuracy: 0.9693 - val_loss: 0.3402 - val_accuracy

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 89.42%


In [19]:
cnn_scores

[0.3819821774959564, 0.8941798806190491]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

71/71 [==============================] - 10s 129ms/step
Accuracy: 89.42%
Precision: 91.15%
Recall: 89.42%
F1 score: 88.91%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-apel+peach.csv', index=False)

+------+----------------------------------------------------------------------------+--------------------------+--------------------------+
|      | Filename                                                                   | True Labels              | Predicted Labels         |
|------+----------------------------------------------------------------------------+--------------------------+--------------------------|
|    0 | b19aef4d-ee91-4768-9520-8fdfee2578b4___FREC_Scab 3396.JPG                  | Apple___Apple_scab       | Apple___Apple_scab       |
|    1 | 5fcb921d-3329-443c-a708-37c3943f5493___FREC_Scab 2987_270deg.JPG           | Apple___Apple_scab       | Apple___Apple_scab       |
|    2 | 14828181-53d0-4dfb-92b1-d7a52602f3cb___RS_HL 7517_flipTB.JPG               | Apple___healthy          | Apple___healthy          |
|    3 | f90bff66-b339-4a6f-acf4-85ed13aebff8___FREC_C.Rust 9843_newGRR.JPG         | Apple___Cedar_apple_rust | Apple___Cedar_apple_rust |
|    4 | 5977222a-8f

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-apel+peach.csv', index=False)

False Predictions:
+------+--------------------------------------------------------------------------+--------------------------+--------------------------+
|      | Filename                                                                 | True Labels              | Predicted Labels         |
|------+--------------------------------------------------------------------------+--------------------------+--------------------------|
|   21 | 93f9f8ea-31c2-412f-b3c4-49ee75312d76___JR_FrgE.S 2978.JPG                | Apple___Black_rot        | Apple___Apple_scab       |
|   23 | 448d1f3f-9794-4148-a180-7246ef9291cf___JR_FrgE.S 2930_270deg.JPG         | Apple___Black_rot        | Apple___Apple_scab       |
|   29 | d3524c88-0d60-43e0-a181-3ecc0facd54a___RS_HL 8181_flipTB.JPG             | Apple___healthy          | Apple___Apple_scab       |
|   32 | f3c88876-d854-4f94-999c-7fcddb87ced7___RS_HL 8202_flipTB.JPG             | Apple___healthy          | Apple___Apple_scab       |
|   39 | 5db8c8

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

284/284 [==============================] - 38s 134ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 281.4099428653717 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

71/71 [==============================] - 8s 111ms/step
Random Forest Classifier Accuracy: 97.13%
Random Forest Classifier Precision: 0.97
Random Forest Classifier Recall: 0.97
Random Forest Classifier F1 Score: 0.97


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-apel+peach.csv', index=False)

+------+----------------------------------------------------------------------------+--------------------------+--------------------------+
|      | Filename                                                                   | True Labels              | Predicted Labels         |
|------+----------------------------------------------------------------------------+--------------------------+--------------------------|
|    0 | b19aef4d-ee91-4768-9520-8fdfee2578b4___FREC_Scab 3396.JPG                  | Apple___Apple_scab       | Apple___Apple_scab       |
|    1 | 5fcb921d-3329-443c-a708-37c3943f5493___FREC_Scab 2987_270deg.JPG           | Apple___Apple_scab       | Apple___Apple_scab       |
|    2 | 14828181-53d0-4dfb-92b1-d7a52602f3cb___RS_HL 7517_flipTB.JPG               | Apple___healthy          | Apple___healthy          |
|    3 | f90bff66-b339-4a6f-acf4-85ed13aebff8___FREC_C.Rust 9843_newGRR.JPG         | Apple___Cedar_apple_rust | Apple___Cedar_apple_rust |
|    4 | 5977222a-8f

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-apel+peach.csv', index=False)

False Predictions:
+------+--------------------------------------------------------------------------+--------------------------+--------------------------+
|      | Filename                                                                 | True Labels              | Predicted Labels         |
|------+--------------------------------------------------------------------------+--------------------------+--------------------------|
|   21 | 93f9f8ea-31c2-412f-b3c4-49ee75312d76___JR_FrgE.S 2978.JPG                | Apple___Black_rot        | Apple___healthy          |
|   32 | f3c88876-d854-4f94-999c-7fcddb87ced7___RS_HL 8202_flipTB.JPG             | Apple___healthy          | Apple___Apple_scab       |
|  156 | db898ee2-3951-4c43-b4a9-577e2ed0453d___Rutg._HL 3674_newPixel25.JPG      | Peach___healthy          | Apple___Cedar_apple_rust |
|  221 | 03d420e7-42a1-41d8-9600-16d577043c64___RS_HL 5738.JPG                    | Apple___healthy          | Apple___Apple_scab       |
|  229 | 9cbb64

In [30]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-apel+peach.csv', index=False)

False Predictions:
+------+--------------------------------------------------------------------------+--------------------------+--------------------------+
|      | Filename                                                                 | True Labels              | Predicted Labels         |
|------+--------------------------------------------------------------------------+--------------------------+--------------------------|
|   21 | 93f9f8ea-31c2-412f-b3c4-49ee75312d76___JR_FrgE.S 2978.JPG                | Apple___Black_rot        | Apple___healthy          |
|   32 | f3c88876-d854-4f94-999c-7fcddb87ced7___RS_HL 8202_flipTB.JPG             | Apple___healthy          | Apple___Apple_scab       |
|  156 | db898ee2-3951-4c43-b4a9-577e2ed0453d___Rutg._HL 3674_newPixel25.JPG      | Peach___healthy          | Apple___Cedar_apple_rust |
|  221 | 03d420e7-42a1-41d8-9600-16d577043c64___RS_HL 5738.JPG                    | Apple___healthy          | Apple___Apple_scab       |
|  229 | 9cbb64